### Введение в обработку естественного языка. Урок 3. Создание признакового пространства.

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
with open('combine_df.pickle', 'rb') as f:
    combine_df = pickle.load(f)

In [3]:
combine_df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love yoyou take with yoyou all the time ...,"[model, love, yoyou, take, with, yoyou, all, t...","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [4]:
import spacy

In [5]:
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 1000000

In [6]:
%%time
combine_df['NER'] = combine_df['clean_tweet'].apply(lambda x:  nlp(x))
ner = combine_df['NER'].tolist()

NER = []
for doc in ner:
    for ent in doc.ents:
        NER.append((ent.text, ent.label_))

df_ner = pd.DataFrame(NER, columns=['text', 'ner'])
df_ner.head()

Wall time: 3min 52s


,text,ner
0,wheelchair vans,ORG
1,pdx,ORG
2,bihday,PERSON
3,tomorrow,DATE
4,the next school year,DATE


In [7]:
# ТОП-20 популярных NER
top_20_ner = df_ner.ner.value_counts().head(20)
top_20_ner

DATE           11502
PERSON          8039
ORG             5572
GPE             4571
TIME            2021
NORP            1450
CARDINAL        1098
ORDINAL          647
FAC              304
LOC              216
EVENT            150
PRODUCT           63
LANGUAGE          43
QUANTITY          34
WORK_OF_ART       27
LAW               21
MONEY              8
PERCENT            3
Name: ner, dtype: int64

In [9]:
# # ТОП-20 популярных персон
top_20_pers = df_ner.loc[df_ner['ner'] == 'PERSON']
top_20_pers = top_20_pers.text.value_counts().head(20)
top_20_pers

hea                                                        117
hu                                                          83
blur sun                                                    54
bihday                                                      46
feminismiscancer feminismisterrorism feminismmuktbharat     40
christina grimmie                                           38
hillary                                                     36
sikh temple                                                 28
tgif ff                                                     26
don                                                         24
detoxdiet altwaystoheal                                     21
clinton                                                     21
ripchristina                                                20
jo cox                                                      20
carl paladino                                               19
regrann                                                

In [10]:
# # ТОП-20 популярных организаций
top_20_org = df_ner.loc[df_ner['ner'] == 'ORG']
top_20_org = top_20_org.text.value_counts().head(20)
top_20_org

bong bing           107
app                  84
gop                  77
islam                58
house                47
social analytics     40
nba                  39
usa                  34
amazon               32
sta                  31
sma                  30
cnn                  28
euro                 26
fed                  25
ios                  23
eu                   22
fbi                  21
sun                  20
bogota colombia      20
congress             19
Name: text, dtype: int64

In [11]:
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\samia\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\samia\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [12]:
ner_nltk = {
    "text": [],
    "label": [],
}

def nltk_ner(tweet):
    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(tweet.title()))):
         if hasattr(chunk, 'label'):
            ner_nltk['text'].append(' '.join(c[0] for c in chunk))
            ner_nltk['label'].append(chunk.label())

In [13]:
combine_df['clean_tweet'].apply(lambda x: nltk_ner(x))

0        None
1        None
2        None
3        None
4        None
         ... 
49154    None
49155    None
49156    None
49157    None
49158    None
Name: clean_tweet, Length: 49159, dtype: object

In [14]:
ner_df_ntlk = pd.DataFrame.from_dict(ner_nltk)
ner_df_ntlk.head()

,text,label
0,Father,PERSON
1,Kids,ORGANIZATION
2,Lyft Credit,PERSON
3,Offer Wheelchair,PERSON
4,Pdx,GPE


In [15]:
ner_df_ntlk.text.value_counts().head(20)

Happy                         1113
New                            528
Yoyour                         501
Model                          500
Love Yoyou Take With Yoyou     497
Father                         494
Good                           421
Love                           367
Will                           347
Great                          272
Us                             262
Bull Up                        246
Orlando                        239
Life                           205
Ready                          191
Sad                            191
Forward                        167
Have                           165
Gorilla Simulator              162
Got                            147
Name: text, dtype: int64

In [16]:
ner_df_ntlk.loc[ner_df_ntlk['label'] == 'ORGANIZATION'].text.value_counts().head(20)

Love Yoyou Take With Yoyou                497
Well Deletetweets                         126
Development Organizations Work Mindset     95
Vicinity Of Their                          89
Fathers                                    79
Vast Expanse Of Mountains                  76
Lot                                        68
House                                      67
Work Conference                            65
Happiness                                  61
City Each Side                             54
Same                                       53
All                                        52
Melancholy Melancholymusic                 46
Love                                       43
Hate                                       37
Weekend                                    37
Into                                       37
No                                         36
Best                                       35
Name: text, dtype: int64

In [17]:
ner_df_ntlk.loc[ner_df_ntlk['label'] == 'PERSON'].text.value_counts().head(20)

Happy                       995
Model                       498
Father                      481
Will                        345
Good                        313
Love                        255
Bull Up                     246
Sad                         169
Forward                     165
Gorilla Simulator           162
Have                        159
Found Way How               130
Delete Old                  130
Ready                       127
Trump                       118
Attack Bull                 115
Aww                         115
Help                        112
Good Bing Bong Bing Bong    107
Hey                         107
Name: text, dtype: int64

#### Библиотека Spacy отработала лучше.